### Get Corpus

In [4]:
import numpy as np
import pandas as pd
import pickle
provider_notes_first_24_FINAL = pd.read_pickle('../Pickles/Provider_Notes_24.p')

### Test Train Split

In [8]:
import re
from sklearn.cross_validation import train_test_split
def remove_nums(data):
    return re.sub(r"\d+", "", data)

X = provider_notes_first_24_FINAL.text
X = X.apply(remove_nums)
y = provider_notes_first_24_FINAL.hospital_expire_flag
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

### Topic Modeling with LDA

In [9]:
## Add stop words
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
stopwordsfile = open('../TextFiles/stopwords.txt', 'r')
stop = []
for line in stopwordsfile:
    if len(line) > 1:
        stop.append(line.strip())
stop += ['.', ',', '(', ')', "'", '"']
# stop += map(str, range(0, 10000))

### Sklearn LDA

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
 
count_vect = CountVectorizer(ngram_range=(1,2), 
#                                  analyzer = partial(regexp_tokenize, 
#                                                     pattern = tokenization_pattern), 
                             token_pattern = '[a-zA-Z][a-zA-Z][a-zA-Z]+',
                             analyzer = 'word',
                             stop_words = stop,
                             max_df = 0.25,
                             min_df = 5
                             )
count_vect.fit(X[:100])
vecs = count_vect.transform(X[:100])

In [72]:
vecs.shape

(100, 1314)

In [14]:
# Create the term-document matrix
# Transpose it so the terms are the rows
ng_vecs = vecs.transpose()
ng_vecs.shape

(1314, 100)

In [17]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(ng_vecs)

In [18]:
id2word = dict((v, k) for k, v in count_vect.vocabulary_.items())

In [16]:
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [19]:
from sklearn.decomposition import LatentDirichletAllocation
# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=1, random_state=0)
lda_tf.fit(vecs)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=1, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [21]:
import pickle
pickle.dump(lda_tf, open("../Pickles/Sklearn_LDA.p", "wb"))
pickle.dump(vecs, open("../Pickles/vecs.p", "wb"))
pickle.dump(count_vect, open("../Pickles/count_vect.p", 'wb'))

### Gensim LDA

In [81]:
def prep_corpus(docs, stopwords=set(), no_below=5, no_above=0.25):
    print('Building dictionary...')
    dictionary = Dictionary(docs)
    stopword_ids = map(dictionary.token2id.get, stopwords)
    dictionary.filter_tokens(stopword_ids)
    dictionary.compactify()
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    dictionary.compactify()
    print 'Done with dictionary!'
    
    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    print 'Done with corpus!'
    return dictionary, corpus

In [82]:
from gensim.corpora import Dictionary, MmCorpus
import funcy as fp
from nltk.tokenize import RegexpTokenizer
three_plus_token = RegexpTokenizer('[a-zA-Z][a-zA-Z][a-zA-Z]+')

In [83]:
docs = [three_plus_token.tokenize(doc.lower()) for doc in X]

In [84]:
len(docs[0])

477

### Add bigrams

In [85]:
# for doc in docs:
#     for i in range(len(doc)-1):
#         doc.append(doc[i] + '_' + doc[i+1])

In [86]:
len(docs[0])

477

In [87]:
dictionary, corpus = prep_corpus(docs, stopwords= stop)

2016-08-26 02:10:01,718 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


Building dictionary...


2016-08-26 02:10:07,256 : INFO : adding document #10000 to Dictionary(51842 unique tokens: [u'dialyzable', u'spkie', u'localizes', u'unreponsivness', u'sweliing']...)
2016-08-26 02:10:12,614 : INFO : built Dictionary(69733 unique tokens: [u'dialyzable', u'spkie', u'fjluid', u'catechols', u'localizes']...) from 13421 documents (total 15635068 corpus positions)
2016-08-26 02:10:12,838 : INFO : discarding 51589 tokens: [(u'dialyzable', 1), (u'spkie', 1), (u'fjluid', 1), (u'catechols', 1), (u'sweliing', 1), (u'circuitry', 1), (u'ultimatley', 1), (u'woody', 2), (u'lasixmg', 1), (u'dasating', 1)]...
2016-08-26 02:10:12,839 : INFO : keeping 17716 tokens which were in no less than 5 and no more than 3355 (=25.0%) documents
2016-08-26 02:10:12,874 : INFO : resulting dictionary: Dictionary(17716 unique tokens: [u'monitoirng', u'intuabated', u'diffusely', u'hemodynamiclly', u'localizes']...)


Done with dictionary!
Building corpus...
Done with corpus!


In [88]:
len(dictionary)

17716

In [89]:
len(corpus)

13421

In [70]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus = corpus, id2word=dictionary, num_topics=50, passes=10)

2016-08-26 00:37:28,959 : INFO : using symmetric alpha at 0.02
2016-08-26 00:37:28,960 : INFO : using symmetric eta at 0.02
2016-08-26 00:37:28,961 : INFO : using serial LDA version on this node
2016-08-26 00:44:18,068 : INFO : running online LDA training, 50 topics, 10 passes over the supplied corpus of 13421 documents, updating model once every 2000 documents, evaluating perplexity every 13421 documents, iterating 50x with a convergence threshold of 0.001000
2016-08-26 00:44:18,069 : INFO : PROGRESS: pass 0, at document #2000/13421
2016-08-26 00:44:33,891 : INFO : merging changes from 2000 documents into a model of 13421 documents
2016-08-26 00:45:11,522 : INFO : topic #26 (0.020): 0.001*history + 0.001*assessed + 0.001*ccu + 0.001*she_was + 0.001*wife + 0.001*hypotension + 0.001*afternoon + 0.001*sicu + 0.001*poor + 0.001*abdominal
2016-08-26 00:45:11,609 : INFO : topic #45 (0.020): 0.001*assessed + 0.001*history + 0.001*hypotension + 0.001*not_assessed + 0.001*daily + 0.001*bleedin

In [71]:
pickle.dump(lda, open("../Pickles/Gensim_LDA_FINAL.p", "wb"))
pickle.dump(dictionary, open("../Pickles/dictionary_FINAL.p", "wb"))
pickle.dump(corpus, open("../Pickles/corpus_FINAL.p", "wb"))

## Run this in iPython NOT Jupyter. Jupyter kernel dies.

#### For Sklearn

In [ ]:
lda_tf = pickle.load(open("Pickles/Sklearn_LDA.p", "rb"))
vecs = pickle.load(open("Pickles/vecs.p", "rb"))
count_vect = pickle.load(open("Pickles/count_vect.p", "rb"))
data = pyLDAvis.sklearn.prepare(lda, vecs, count_vect)

#### For Gensim

In [62]:
lda = pickle.load(open("Pickles/Gensim_LDA_FINAL.p", "rb"))
dictionary = pickle.load(open("Pickles/dictionary_FINAL.p", "rb"))
corpus = pickle.load(open("Pickles/corpus_FINAL.p", "rb"))

In [ ]:
import pyLDAvis.gensim
import pyLDAvis.sklearn
data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [26]:
pickle.dump(data, "Pickles/LDAvis_FINAL.data")